In [1]:
import sqlite3
import pandas as pd

# Connect to a database (or create one if it doesn't exist)
conn = sqlite3.connect('final.db')
c = conn.cursor()

# Create tables
c.execute('''
    CREATE TABLE IF NOT EXISTS Passengers (
        id INTEGER NOT NULL,
        FirstName TEXT NOT NULL,
        Lastname TEXT NOT NULL,
        Nationality TEXT NOT NULL,
        Passport TEXT NOT NULL,
        Age INTEGER NOT NULL,
        PRIMARY KEY (id)

    )
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS Airlines (
        id INTEGER NOT NULL,
        name TEXT NOT NULL,
        PRIMARY KEY (id)
    )
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS Aircrafts (
        id INTEGER NOT NULL,
        Model TEXT NOT NULL,
        AirlineID INTEGER NOT NULL,
        Capacity INTEGER NOT NULL,
        PRIMARY KEY (id),
        FOREIGN KEY (AirlineID) REFERENCES Airlines (id)
    )
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS CrewMembers (
        id INTEGER NOT NULL,
        FirstName TEXT NOT NULL,
        LastName TEXT NOT NULL,
        Role TEXT NOT NULL,
        AirlineID INTEGER NOT NULL,
        FlightID INTEGER NOT NULL,
        PRIMARY KEY (id),
        FOREIGN KEY (AirlineID) REFERENCES Airlines (id),
        FOREIGN KEY (FlightID) REFERENCES Flights (id)
    )
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS Airports (
        id INTEGER NOT NULL,
        Name TEXT NOT NULL,
        Location TEXT NOT NULL,
        Abbreviation TEXT NOT NULL,
        PRIMARY KEY (id)
    )
''')

c.execute('''
    CREATE TABLE IF NOT EXISTS AircraftLocation (
        AircraftID INTEGER NOT NULL,
        AirportID INTEGER NOT NULL,
        PRIMARY KEY (AircraftID, AirportID),
        FOREIGN KEY (AircraftID) REFERENCES Aircrafts (id),
        FOREIGN KEY (AirportID) REFERENCES Airports (id)
    )
''')


c.execute('''
    CREATE TABLE IF NOT EXISTS Flights (
        id INTEGER NOT NULL,
        AirlineID INTEGER NOT NULL,
        AircraftID INTEGER NOT NULL,
        DepartureTime TIMESTAMP NOT NULL,
        ArrivalTime TIMESTAMP NOT NULL,
        OriginAirportID INTEGER NOT NULL,
        DestinationAirportID INTEGER NOT NULL,
        PRIMARY KEY (id),
        FOREIGN KEY (AirlineID) REFERENCES Airlines (id),
        FOREIGN KEY (AircraftID) REFERENCES Aircrafts (id),
        FOREIGN KEY (OriginAirportID) REFERENCES Airports (id),
        FOREIGN KEY (DestinationAirportID) REFERENCES Airports (id)
    )
''')

# Automatic Delete when the linked flight is delete
c.execute('''
    CREATE TABLE IF NOT EXISTS Baggage (
        id INTEGER NOT NULL,
        Weight DOUBLE NOT NULL,
        Status TEXT NOT NULL,
        PassengerID INTEGER NOT NULL,
        FlightID INTEGER NOT NULL,
        PRIMARY KEY (id),
        FOREIGN KEY (PassengerID) REFERENCES Passengers (id),
        FOREIGN KEY (FlightID) REFERENCES Flights (id) ON DELETE CASCADE
    )
''')

# Automatic Delete when the linked flight is delete
c.execute('''
    CREATE TABLE IF NOT EXISTS Booking (
        id INTEGER NOT NULL,
        FlightID INTEGER NOT NULL,
        PassengerID INTEGER NOT NULL,
        Seat TEXT NOT NULL,
        BookingDate DATE NOT NULL,
        TicketStatus TEXT NOT NULL,
        Price INTEGER NOT NULL,
        PRIMARY KEY (id),
        FOREIGN KEY (FlightID) REFERENCES Flights (id) ON DELETE CASCADE,
        FOREIGN KEY (PassengerID) REFERENCES Passengers (id)
    )
''')

conn.commit()
conn.close()


In [2]:
from faker import Faker
from random import randint

fake = Faker()

# Assuming you have already connected to the database and have a cursor (conn and c from the previous code)
conn = sqlite3.connect('final.db')
c = conn.cursor()

nbAirline = 15
nbAircraft = 100
nbAirport = 10
nbPassengers = 10000
nbBooking = nbPassengers * 10
nbFlights = int(nbPassengers / 10)
nbCrewMembers = 200
nbBaggage = nbBooking

# Insert data into Aircrafts table
items = []
for i in range(nbAircraft):
    items.append((i + 1, "Aircraft" + str(i + 1), randint(1, nbAirline), randint(100, 500)))

c.executemany('''
    INSERT INTO Aircrafts (id, Model, AirlineID, Capacity)
    VALUES (?, ?, ?, ?)
''', items)

# Insert data into Airlines table
items = []
for i in range(nbAirline):
    items.append((i + 1, fake.company()))

c.executemany('''
    INSERT INTO Airlines (id, name)
    VALUES (?, ?)
''', items)

# Insert data into Airports table
items = []
for i in range(nbAirport):
    city = fake.city()
    items.append((i + 1, fake.address(), city, city[0:3].upper()))

c.executemany('''
    INSERT INTO Airports (id, Location, Name, Abbreviation)
    VALUES (?, ?, ?, ?)
''', items)

# Insert data into Baggage table
items = []
for i in range(nbBaggage):
    items.append((i + 1, randint(1, 50), ["Checked", "Unchecked", "Lose"][randint(0, 2)], randint(1, nbPassengers), randint(1, nbFlights)))

c.executemany('''
    INSERT INTO Baggage (id, Weight, Status, PassengerID, FlightID)
    VALUES (?, ?, ?, ?, ?)
''', items)

# Insert data into Booking table
items = []
for i in range(nbBooking):
    items.append((i + 1, randint(1, nbFlights), randint(1, nbPassengers), "ABCDEFGHIJKLMNOPQRSTUVWXYZ"[randint(0, 25)] + str(randint(1, 9)), fake.date_between(start_date='-1y', end_date='+1y'), ["Confirmed", "Cancelled"][randint(0, 1)], randint(600, 1000)))

c.executemany('''
    INSERT INTO Booking (id, FlightID, PassengerID, Seat, BookingDate, TicketStatus, Price)
    VALUES (?, ?, ?, ?, ?, ?, ?)
''', items)

# Insert data into CrewMembers table
items = []
for i in range(nbCrewMembers):
    items.append((i + 1, fake.first_name(), fake.last_name(), ["Pilot", "Co-Pilot", "Flight Attendant", "Engineer", "Navigator"][randint(0, 4)], randint(1, nbAirline), randint(1, nbFlights)))

c.executemany('''
    INSERT INTO CrewMembers (id, FirstName, LastName, Role, AirlineID, FlightID)
    VALUES (?, ?, ?, ?, ?, ?)
''', items)

# Insert data into Flights table
items = []
for i in range(nbFlights):
    items.append((i + 1, randint(1, nbAirline), randint(1, nbAircraft), fake.date_time_between(start_date='-1y', end_date='+1y'), fake.date_time_between(start_date='-1y', end_date='+1y'), randint(1, nbAirport), randint(1, nbAirport)))

c.executemany('''
    INSERT INTO Flights (id, AirlineID, AircraftID, DepartureTime, ArrivalTime, OriginAirportID, DestinationAirportID)
    VALUES (?, ?, ?, ?, ?, ?, ?)
''', items)

# Insert data into Passengers table
items = []
for i in range(nbPassengers):
    items.append((i + 1, fake.first_name(), fake.last_name(), fake.country(), fake.ssn(), randint(1, 100)))

c.executemany('''
    INSERT INTO Passengers (id, FirstName, Lastname, Nationality, Passport, Age)
    VALUES (?, ?, ?, ?, ?, ?)
''', items)

# Insert data into AircraftLocation table
items = []
for i in range(nbAircraft):
    items.append((i + 1, randint(1, nbAirport)))

c.executemany('''
    INSERT INTO AircraftLocation (AircraftID, AirportID)
    VALUES (?,?)
''', [(1, 1), (2, 2), (3, 3), (4, 4), (5, 5)])

# Commit the change
conn.commit()
conn.close()


In [3]:
#by Mahe

# Connect to the SQLite database
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Rank flights by price using WINDOW FUNCTION RANK
ranked_flight_query = """
    SELECT
        FlightID,
        Price,
        RANK() OVER (ORDER BY price ASC) as flight_rank
    FROM
        Booking
"""

ranked_flight_df = pd.read_sql_query(ranked_flight_query, conn)

# Close the database connection
conn.close()
# Display the result
ranked_flight_df

,FlightID,Price,flight_rank
0,838,600,1
1,742,600,1
2,105,600,1
3,860,600,1
4,280,600,1
...,...,...,...
99995,708,1000,99749
99996,14,1000,99749
99997,86,1000,99749
99998,725,1000,99749


In [4]:
#by Mahe

# Connect to the SQLite database
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Calculate the average price for each company using the ranked flights
average_company_query = """
    SELECT
        c.id,
        c.name,
        AVG(b_ranked.Price) as average_price
    FROM
        Airlines c
    JOIN
        Flights f ON c.id = f.AirlineID
    JOIN
        (SELECT FlightId, Price FROM Booking ORDER BY Price ASC) b_ranked ON f.id = b_ranked.FlightId
    GROUP BY
        c.id;
"""

average_company_df = pd.read_sql_query(average_company_query, conn)

# Close the database connection
conn.close()
# Display the result
average_company_df

,id,name,average_price
0,1,Williams-Gomez,800.032527
1,2,Walker PLC,796.956161
2,3,Wilson LLC,798.595509
3,4,Hughes-Davis,801.688098
4,5,Duran-Jennings,800.954153
5,6,"Hopkins, Johnson and Thompson",799.570455
6,7,"Morrison, York and English",800.252939
7,8,"Cunningham, Adams and Howard",801.309013
8,9,"Tucker, Thornton and Holden",801.306977
9,10,Hodges Ltd,800.754947


In [5]:
#by Mahe

# Connect to the SQLite database
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Calculate the average price for each company using the ranked flights
baggage_count_query = """
    SELECT
        b.FlightId,
        COUNT(b.id) as baggage_count
    FROM
        Baggage b
    GROUP BY
        b.FlightId
"""

baggage_count_df = pd.read_sql_query(baggage_count_query, conn)


# Close the database connection
conn.close()
# Display the result
baggage_count_df

,FlightID,baggage_count
0,1,89
1,2,105
2,3,100
3,4,87
4,5,86
...,...,...
995,996,101
996,997,88
997,998,103
998,999,99


In [6]:
#By Mahe

# Connect to the SQLite database
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# SQL query to retrieve flight information, including average price, baggage count, departure city and date, and arrival city and date
flight_info_query = """
    SELECT
        f.id,
        dep_airport.name as departure_airport,
        arr_airport.name as arrival_airport,
        f.DepartureTime,
        f.ArrivalTime,
        c.name,
        AVG(b.Price) as average_price,
        COUNT(DISTINCT bg.id) as baggage_count,
        RANK() OVER (ORDER BY AVG(b.price) ASC) as flight_rank
    FROM
        Flights f
    JOIN
        Airlines c ON f.AirlineID = c.id
    JOIN
        Airports dep_airport ON f.OriginAirportID = dep_airport.id
    JOIN
        Airports arr_airport ON f.DestinationAirportID = arr_airport.id
    LEFT JOIN
        Booking b ON f.id = b.FlightId
    LEFT JOIN
        Baggage bg ON f.id = bg.FlightId
    GROUP BY
        f.id
"""

# Use pandas to execute the SQL query and read the result into a DataFrame
flight_info_df = pd.read_sql_query(flight_info_query, conn)

# Close the database connection
conn.close()

# Display the result (the flight information DataFrame)
flight_info_df

,id,departure_airport,arrival_airport,DepartureTime,ArrivalTime,name,average_price,baggage_count,flight_rank
0,82,Curtisside,Lake Casey,2023-01-04 23:09:51,2024-07-25 15:36:34,"Cunningham, Adams and Howard",770.831683,87,1
1,328,Lake Mark,Lake Mark,2023-01-26 11:13:31,2024-11-04 15:42:13,Hughes-Davis,771.479592,122,2
2,765,Woodardburgh,Doylemouth,2024-01-01 04:06:09,2024-08-28 09:55:59,"Martinez, Rodriguez and Moore",771.767677,104,3
3,742,Lake Mark,Stevenfort,2024-09-20 22:04:23,2022-12-18 09:10:24,"Davis, Schultz and Cox",772.556962,102,4
4,387,Curtisside,Lake Mark,2023-09-28 07:47:43,2024-02-26 09:34:13,Duran-Jennings,772.961905,104,5
...,...,...,...,...,...,...,...,...,...
995,340,Curtisside,Solismouth,2023-12-14 06:50:20,2023-01-08 12:01:28,Ramirez PLC,831.287356,120,996
996,689,Lake Mark,Stevenfort,2024-04-14 08:14:54,2024-09-20 22:12:53,Hodges Ltd,832.896907,96,997
997,373,Lake Mark,Woodardburgh,2024-11-25 11:58:09,2023-11-17 05:12:13,Ramirez PLC,833.295918,86,998
998,618,Doylemouth,Woodardburgh,2023-10-02 06:03:29,2023-06-23 17:56:56,"Morrison, York and English",838.213333,99,999


In [7]:
#Cyprien SINGEZ

# Advanced °1
# Print all the adult passengers (+18) and order them by number of flights taken (CTE)

import pandas as pd

conn = sqlite3.connect('final.db')

df = pd.read_sql_query('''
    WITH cte AS (
        SELECT
            id,
            FirstName,
            LastName,
            Nationality
        FROM
            Passengers
        WHERE
            Age >= 18
    )
    SELECT
        cte.FirstName,
        cte.LastName,
        cte.Nationality,
        COUNT(*) AS TotalFlights
    FROM
        cte
    JOIN
        Booking ON cte.id = Booking.PassengerID
    GROUP BY
        cte.id, cte.FirstName, cte.LastName, cte.Nationality
    ORDER BY
        TotalFlights DESC;
''', conn)

conn.close()

df


,FirstName,LastName,Nationality,TotalFlights
0,Desiree,Dorsey,Qatar,24
1,Scott,Hansen,Peru,23
2,Eduardo,Morgan,Bosnia and Herzegovina,22
3,Ralph,James,Guinea,22
4,Matthew,Pace,Faroe Islands,22
...,...,...,...,...
8314,Jonathan,Bell,Mauritius,2
8315,Shelby,Santos,Azerbaijan,2
8316,Yvonne,Perez,Burkina Faso,2
8317,Shelly,Acosta,Qatar,1


In [8]:
# Cyprien SINGEZ

# Advanced °2
# Print the final destination of a bagage (COMPLEX JOINS)

import pandas as pd

conn = sqlite3.connect('final.db')

df = pd.read_sql_query('''
    SELECT
        FirstName,
        LastName,
        Weight,
        Name Destination_Airport
    FROM Baggage
    INNER JOIN
        Flights
            ON Baggage.FlightID = Flights.id
    INNER JOIN
        Airports
            ON Airports.id = Flights.DestinationAirportID
    INNER JOIN
        Passengers
            ON Baggage.PassengerID = Passengers.id
''', conn)

conn.close()

df


,FirstName,Lastname,Weight,Destination_Airport
0,Mary,Bowman,41.0,Solismouth
1,Peggy,Cabrera,32.0,Stevenfort
2,Kimberly,Gray,44.0,Stevenfort
3,Thomas,Dixon,46.0,Woodardburgh
4,Savannah,Hall,41.0,Stevenfort
...,...,...,...,...
99995,Sheila,Payne,25.0,Lake Mark
99996,Anthony,Wagner,21.0,Curtisside
99997,Billy,Sandoval,42.0,Stevenfort
99998,Kimberly,Thompson,22.0,Lake Mark


In [9]:
# Cyprien SINGEZ

# Advanced °3
# Rank companies by number of aircrafts (WINDOW FUNCTION)

import pandas as pd

conn = sqlite3.connect('final.db')

df = pd.read_sql_query('''
    SELECT
        RANK() OVER (ORDER BY (SELECT COUNT(*) FROM Aircrafts WHERE AirlineID = Airlines.id) DESC) as Rank,
        (SELECT COUNT(*) FROM Aircrafts WHERE AirlineID = Airlines.id) as NbAircraft,
        Name
    FROM Airlines
''', conn)

conn.close()

df


,Rank,NbAircraft,name
0,1,11,Walker PLC
1,2,9,Wilson LLC
2,3,8,Duran-Jennings
3,3,8,Harrison PLC
4,5,7,"Morrison, York and English"
5,5,7,Hodges Ltd
6,5,7,"Davis, Schultz and Cox"
7,5,7,"Martinez, Rodriguez and Moore"
8,5,7,"Glass, Daniels and Glenn"
9,10,6,Hughes-Davis


In [10]:
# Cyprien SINGEZ

# Intermediate °1
# Select all distinct different airlines that the passenger with specific ID has flown with (SUBQUERY)

import pandas as pd

conn = sqlite3.connect('final.db')

passengerID = 5

df = pd.read_sql_query(f'''
    SELECT
        Name
    FROM
        Airlines
    WHERE
        id IN (SELECT AirlineID FROM Flights WHERE id IN (SELECT FlightID FROM Booking WHERE PassengerID = { passengerID }))
    ORDER BY Name ASC
''', conn)

conn.close()

df


,name
0,"Davis, Schultz and Cox"
1,Harrison PLC
2,Hodges Ltd
3,Hughes-Davis
4,"Martinez, Rodriguez and Moore"
5,"Tucker, Thornton and Holden"
6,Williams-Gomez


In [11]:
# Cyprien SINGEZ

# Intermediate °2
# Print the number of flights for each company (JOIN)

import pandas as pd

conn = sqlite3.connect('final.db')

df = pd.read_sql_query('''
    SELECT
        Name,
        NbFlights
    FROM
        Airlines
    INNER JOIN
        (SELECT
            AirlineID,
            COUNT(*) as NbFlights
        FROM
            Flights
        GROUP BY
            AirlineID) as FlightsPerAirline
        ON Airlines.id = FlightsPerAirline.AirlineID
    ORDER BY
        NbFlights DESC
''', conn)

conn.close()

df


,name,NbFlights
0,Duran-Jennings,81
1,Williams-Gomez,78
2,"Tucker, Thornton and Holden",74
3,Hughes-Davis,73
4,"Davis, Schultz and Cox",68
5,Ramirez PLC,68
6,"Hopkins, Johnson and Thompson",67
7,"Morrison, York and English",67
8,Hodges Ltd,65
9,"Glass, Daniels and Glenn",65


In [12]:
# Cyprien SINGEZ

# Intermediate °3
# Print the total amount of money spent per each passenger in descending order (SUM + SUBQUERY + GROUP BY)

import pandas as pd

conn = sqlite3.connect('final.db')

df = pd.read_sql_query('''
    SELECT
        (SELECT FirstName FROM Passengers WHERE id = PassengerID) as FirstName,
        (SELECT LastName FROM Passengers WHERE id = PassengerID) as FirstName,
        SUM(Price) as TotalMoneySpent
    FROM
        Booking
    GROUP BY
        PassengerID
    ORDER BY
        SUM(Price) DESC
''', conn)

conn.close()

df


,FirstName,FirstName,TotalMoneySpent
0,Desiree,Dorsey,19818
1,Katrina,Wagner,19420
2,Scott,Hansen,18742
3,Lisa,Hall,17815
4,William,Cannon,17676
...,...,...,...
9993,Shelby,Santos,1304
9994,Diane,Lopez,1257
9995,Shelly,Acosta,838
9996,Jasmine,Salazar,816


In [13]:
# Done by Antoine Van Gorp

# Connect to the SQLite database named 'final.db'
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the SQL query to retrieve information about passengers, including their first and last names,
# and the departure location of the flights they booked
pass_loc = """
    SELECT
        Passengers.FirstName,
        Passengers.Lastname,
        Airports.Location AS DepartureLocation
    FROM
        Passengers
    JOIN
        Booking ON Passengers.id = Booking.PassengerID,
        Flights ON Booking.FlightID = Flights.id,
        Airports ON Flights.OriginAirportID = Airports.id
"""

# Execute the SQL query and store the result in a Pandas DataFrame
pass_loc_df = pd.read_sql_query(pass_loc, conn)

# Close the database connection since the query has been executed
conn.close()

# Display the resulting DataFrame
pass_loc_df


,FirstName,Lastname,DepartureLocation
0,Mark,Scott,"PSC 6888, Box 9686\nAPO AE 77033"
1,Michael,Moreno,"07398 Gina Stream\nDevinport, GA 83593"
2,Kendra,Elliott,"07398 Gina Stream\nDevinport, GA 83593"
3,Shawn,Williams,"910 Kramer Overpass\nNicholasberg, MP 02626"
4,Jill,Bryan,"6594 Gross Ford Apt. 305\nEast James, CT 15562"
...,...,...,...
99995,Sabrina,Mcmahon,"396 Paul Crossroad\nClarkchester, MS 83484"
99996,Travis,Phillips,"66778 Harvey Dam\nEast Elizabethport, WY 87445"
99997,Leslie,Baldwin,"82426 Cooper Island\nSouth Julieview, NV 92565"
99998,Brooke,Medina,"PSC 6888, Box 9686\nAPO AE 77033"


In [14]:
# Done by Antoine Van Gorp

# Connect to the SQLite database named 'final.db'
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the SQL query to retrieve information about remaining seats on flights
aircraft_seats = """
    SELECT
        Flights.id AS FlightID,
        Aircrafts.Capacity - COUNT(Booking.id) AS RemainingSeats,
        COUNT(Booking.id) AS SeatsTaken
    FROM
        Flights
    JOIN
        Aircrafts ON Flights.AircraftID = Aircrafts.id
    LEFT JOIN
        Booking ON Flights.id = Booking.FlightID
    GROUP BY
        Flights.id
"""

# Execute the SQL query and store the result in a Pandas DataFrame
aircraft_seats_df = pd.read_sql_query(aircraft_seats, conn)

# Close the database connection since the query has been executed
conn.close()

# Display the resulting DataFrame
aircraft_seats_df


,FlightID,RemainingSeats,SeatsTaken
0,1,297,94
1,2,314,105
2,3,260,102
3,4,99,108
4,5,59,104
...,...,...,...
995,996,300,113
996,997,182,93
997,998,365,95
998,999,375,116


In [15]:
# Done by Antoine Van Gorp
# Connect to the SQLite database named 'final.db'
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the SQL query to retrieve information about aircraft and their corresponding airports
aircraft_airport = """
    SELECT
        Airports.name,
        Aircrafts.Model
    FROM
        AircraftLocation
    JOIN
        Airports on Airports.id = AircraftLocation.AirportID,
        Aircrafts on Aircrafts.id = AircraftLocation.AircraftID
    ORDER BY
        Airports.name
"""

# Execute the SQL query and store the result in a Pandas DataFrame
aircraft_airport_df = pd.read_sql_query(aircraft_airport, conn)

# Close the database connection since the query has been executed
conn.close()

# Display the resulting DataFrame
aircraft_airport_df


,Name,Model
0,Doylemouth,Aircraft4
1,Lake Mark,Aircraft1
2,Lake Stacyfort,Aircraft2
3,West Michael,Aircraft3
4,Woodardburgh,Aircraft5


In [16]:
# Done by Antoine Van Gorp

# Connect to the SQLite database named 'final.db'
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the SQL query to retrieve information about flight departures,
# including the abbreviation of the origin airport, departure time,
# and the rank of departure times using the RANK() window function
departur_rank = """
    SELECT
        Airports.Abbreviation AS OriginAirport,
        Flights.DepartureTime,
        RANK() OVER (ORDER BY Flights.DepartureTime) AS DepartureTimeRank
    FROM
        Flights
    JOIN
        Airports ON Airports.id = Flights.OriginAirportID
"""

# Execute the SQL query and store the result in a Pandas DataFrame
departur_rank_df = pd.read_sql_query(departur_rank, conn)

# Close the database connection since the query has been executed
conn.close()

# Display the resulting DataFrame
departur_rank_df


,OriginAirport,DepartureTime,DepartureTimeRank
0,DOY,2022-11-30 11:15:01,1
1,WES,2022-11-30 14:31:47,2
2,DOY,2022-12-01 10:08:48,3
3,CUR,2022-12-02 11:26:16,4
4,WES,2022-12-02 12:48:29,5
...,...,...,...
995,DOY,2024-11-24 21:30:04,996
996,LAK,2024-11-25 02:26:20,997
997,LAK,2024-11-25 11:58:09,998
998,CUR,2024-11-26 08:49:00,999


In [17]:
# Done by Antoine Van Gorp

# Connect to the SQLite database named 'final.db'
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the SQL query to retrieve information about passengers, their baggage weight,
# and assign a rank to each row based on baggage weight in descending order
pass_bag_waight = """
    SELECT
        Passengers.FirstName,
        Passengers.Lastname,
        Baggage.Weight,
        RANK() OVER (ORDER BY Baggage.Weight DESC) AS BaggageRank
    FROM
        Passengers
    JOIN
        Baggage ON Passengers.id = Baggage.PassengerID
"""

# Execute the SQL query and store the result in a Pandas DataFrame
pass_bag_waight_df = pd.read_sql_query(pass_bag_waight, conn)

# Close the database connection since the query has been executed
conn.close()

# Display the resulting DataFrame
pass_bag_waight_df


,FirstName,Lastname,Weight,BaggageRank
0,Caroline,Montgomery,50.0,1
1,Amanda,Moore,50.0,1
2,Terri,Rodriguez,50.0,1
3,Kyle,Gomez,50.0,1
4,James,Hunter,50.0,1
...,...,...,...,...
99995,Randy,Howard,1.0,97933
99996,Jasmine,Moore,1.0,97933
99997,Christopher,Johnson,1.0,97933
99998,William,Walker,1.0,97933


In [18]:
# Done by Antoine Van Gorp
# Connect to the SQLite database named 'final.db'
conn = sqlite3.connect('final.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Define the SQL query to retrieve information about remaining seats on flights
aircraft_seats = """
    SELECT
        Flights.id AS FlightID,
        Aircrafts.Capacity - COUNT(Booking.id) AS RemainingSeats,
        COUNT(Booking.id) AS SeatsTaken
    FROM
        Flights
    JOIN
        Aircrafts ON Flights.AircraftID = Aircrafts.id
    LEFT JOIN
        Booking ON Flights.id = Booking.FlightID
    GROUP BY
        Flights.id
"""

# Execute the SQL query and store the result in a Pandas DataFrame
aircraft_seats_df = pd.read_sql_query(aircraft_seats, conn)

# Close the database connection since the query has been executed
conn.close()

# Display the resulting DataFrame
aircraft_seats_df



,FlightID,RemainingSeats,SeatsTaken
0,1,297,94
1,2,314,105
2,3,260,102
3,4,99,108
4,5,59,104
...,...,...,...
995,996,300,113
996,997,182,93
997,998,365,95
998,999,375,116


In [19]:
# Clément Monteiro ###### Intermediate 1

# Use a JOIN to print the Airline of the Aircraft

conn = sqlite3.connect('final.db')

query = '''
  SELECT Model, name
  FROM Flights
  INNER JOIN Airlines
  ON Flights.AirlineID=Airlines.id
  INNER JOIN Aircrafts
  ON Flights.AircraftID=Aircrafts.id
'''

query_o = pd.read_sql_query(query, conn)
conn.close()
query_o

,Model,name
0,Aircraft8,Ramirez PLC
1,Aircraft88,Hodges Ltd
2,Aircraft58,"Glass, Daniels and Glenn"
3,Aircraft5,Ramirez PLC
4,Aircraft72,"Glass, Daniels and Glenn"
...,...,...
995,Aircraft13,"Martinez, Rodriguez and Moore"
996,Aircraft75,"Tucker, Thornton and Holden"
997,Aircraft94,"Glass, Daniels and Glenn"
998,Aircraft53,Harrison PLC


In [20]:
# Clément Monteiro ###### Intermediate 2

# Use a GROUP BY AND A COUNT to print the number of crew members by role
conn = sqlite3.connect('final.db')

query = '''
  SELECT role,
  Count() Number_of_crew
  FROM CrewMembers
  GROUP BY role
'''

query_o = pd.read_sql_query(query, conn)
conn.close()
query_o

,Role,Number_of_crew
0,Co-Pilot,49
1,Engineer,41
2,Flight Attendant,28
3,Navigator,37
4,Pilot,45


In [21]:
# Clément Monteiro ###### Intermediate 3

# Use a JOIN a GROUP BY and a COUNT to print the number of aircraft by company

conn = sqlite3.connect('final.db')

query = '''
  SELECT name,
  COUNT() Number_of_Aircraft
  FROM Aircrafts
  INNER JOIN Airlines
  ON Aircrafts.AirlineID=Airlines.id
  GROUP BY AirlineID 
'''

query_o = pd.read_sql_query(query, conn)
conn.close()
query_o

,name,Number_of_Aircraft
0,Williams-Gomez,5
1,Walker PLC,11
2,Wilson LLC,9
3,Hughes-Davis,6
4,Duran-Jennings,8
5,"Hopkins, Johnson and Thompson",4
6,"Morrison, York and English",7
7,"Cunningham, Adams and Howard",3
8,"Tucker, Thornton and Holden",5
9,Hodges Ltd,7


In [22]:
# Clément Monteiro ###### Advanced 1

# Use a COMPLEX JOIN to print the name of the Origin airport by Booking

conn = sqlite3.connect('final.db')

query = '''SELECT FirstName, LastName, BookingDate, Name FROM Booking
   INNER JOIN
   Flights
   ON Booking.FlightID = Flights.id
   INNER JOIN
   Airports
   ON Airports.id = Flights.OriginAirportID
   INNER JOIN
   Passengers
   ON Booking.PassengerID = Passengers.id'''


query_o = pd.read_sql_query(query, conn)
conn.close()
query_o

,FirstName,Lastname,BookingDate,Name
0,Mark,Scott,2024-11-24,Woodardburgh
1,Michael,Moreno,2023-11-25,Stevenfort
2,Kendra,Elliott,2023-11-19,Stevenfort
3,Shawn,Williams,2023-05-28,Lake Casey
4,Jill,Bryan,2024-01-06,Solismouth
...,...,...,...,...
99995,Sabrina,Mcmahon,2023-11-01,Curtisside
99996,Travis,Phillips,2024-09-08,West Amandahaven
99997,Leslie,Baldwin,2023-04-15,Doylemouth
99998,Brooke,Medina,2024-07-28,Woodardburgh


In [23]:
# Clément Monteiro ###### Advanced 2

# Use a WINDOW RANK to rank the airport by number of aircraft

conn = sqlite3.connect('final.db')

query = ''' SELECT name,
    RANK()
    OVER (ORDER BY COUNT() DESC) Ranking
  FROM AircraftLocation
    INNER JOIN
    Airports
    ON AircraftLocation.AirportID=Airports.id
  GROUP BY AirportID'''

query_o = pd.read_sql_query(query, conn)
conn.close()
query_o

,Name,Ranking
0,Woodardburgh,1
1,Doylemouth,1
2,West Michael,1
3,Lake Stacyfort,1
4,Lake Mark,1


In [24]:
# Clément Monteiro ###### Advanced 3

# Rank passenger with specific id most taken companies (WINDOW FUNCTION)

conn = sqlite3.connect('final.db')

passengerID = 5

query_o = pd.read_sql_query(f'''
    SELECT
        RANK()
        OVER (ORDER BY (
          SELECT COUNT(*)
          FROM Flights
          WHERE AirlineID = Airlines.id
          AND id IN (
            SELECT FlightID
            FROM Booking WHERE PassengerID = { passengerID })) DESC) as Rank,
        (SELECT COUNT(*)
        FROM Flights
        WHERE AirlineID = Airlines.id
        AND id IN (
          SELECT FlightID
          FROM Booking
          WHERE PassengerID = { passengerID })) as NbTakenFlights,
        Name
    FROM
        Airlines
''', conn)

conn.close()

query_o

,Rank,NbTakenFlights,name
0,1,2,Williams-Gomez
1,2,1,Hughes-Davis
2,2,1,"Tucker, Thornton and Holden"
3,2,1,Hodges Ltd
4,2,1,"Davis, Schultz and Cox"
5,2,1,"Martinez, Rodriguez and Moore"
6,2,1,Harrison PLC
7,8,0,Walker PLC
8,8,0,Wilson LLC
9,8,0,Duran-Jennings
